In [ ]:
import os
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine, text
import time
import numpy as np
import dotenv

dotenv.load_dotenv(".env.local")

from test_unit.tasks_unit1 import UnitTask1
from mrp_functions.base_tasks import TaskBase
from mrp_functions import logging
import re

np.__version__

part_group=os.environ['PART_GROUP']
contract_code=os.environ['CONTRACT_CODE']
mrp_run_date=os.environ['MRP_RUN_DATE']
msc_code=os.environ['MSC_CODE']
plant_code=os.environ['PLANT_CODE']
production_plan_id=int(os.environ['PRODUCTION_PLAN_ID'])
user_code=int(os.environ['USER_CODE'])
simulation=bool(int(os.environ['SIMULATION']))

tb = TaskBase()
tb.db_connection_health_check()

In [ ]:
# Get list of all MSC Code to run MRP
list_msc_plant_code = tb.query_list_distinct_msc_from_production_plan(production_plan_id)
logging.info(f'Number of MSCs from Production Plan: {len(list_msc_plant_code)}')

In [ ]:
# Get end date of production plan
end_date_of_prod_plan = tb.query_end_date_from_production_plan(production_plan_id)
if end_date_of_prod_plan is None:
    raise ValueError("Production Plan End Date is None!") # FIXME: Xu ly nhu nao cho nay ?
# Get list of all date from N to N+6 to end of production Plan
list_mrp_date = tb.query_list_date_from_Nplus1_to_Nplus6_definition(mrp_run_date,
                                                                                end_date_of_prod_plan)
logging.info(f"[{production_plan_id}_{mrp_run_date}] "
             f"System Run From: {list_mrp_date[0]} To: {list_mrp_date[-1]}")

In [ ]:
parent_start_timestamp = time.time()
list_distinct_plant_code = []
for _, _, _plant_code in list_msc_plant_code:
    if _plant_code not in list_distinct_plant_code:
        # To delete Forecast Logical Inventory from current date and plant code
        logging.info(f"[{production_plan_id}_{mrp_run_date}_{_plant_code}] "
                     f"Clear logical inventory Forecast - [Simulation]")
        tb.clear_logical_inventory_forecast(mrp_run_date, _plant_code, True)
        if not simulation:
            logging.info(f"[{production_plan_id}_{mrp_run_date}_{_plant_code}] "
                         f"Clear logical inventory Forecast - [Actual]")
            tb.clear_logical_inventory_forecast(mrp_run_date, _plant_code, False)
        logging.info(f"[{production_plan_id}_{mrp_run_date}_{_plant_code}] "
                     f"Clear logical inventory Forecast finishes after: "
                     f"{time.time() - parent_start_timestamp:.2f} sec")

        # To delete Forecast Results from current date, import id and plant code
        logging.info(f"[{production_plan_id}_{mrp_run_date}_{_plant_code}] "
                     f"Clear MRP Result - [Simulation: {simulation}]")
        tb.clear_mrp_results_forecast(mrp_run_date, production_plan_id, _plant_code, simulation)
        logging.info(f"[{production_plan_id}_{mrp_run_date}_{_plant_code}] "
                     f"Clear MRP Result finishes after: {time.time() - parent_start_timestamp:.2f} sec")

        # To delete future Shortage Parts Quantity from current date and mapped to this production plan
        if simulation:
            logging.info(f"[{production_plan_id}_{mrp_run_date}_{_plant_code}] "
                         f"Clear Shortage parts forecast")
            tb.clear_shortage_parts_forecast(mrp_run_date, production_plan_id, _plant_code)
            logging.info(f"[{production_plan_id}_{mrp_run_date}_{_plant_code}] "
                         f"Clear Shortage parts forecast finishes after: "
                         f"{time.time() - parent_start_timestamp:.2f} sec")

        list_distinct_plant_code.append(_plant_code)
# Close DB Connection because we don't need it anymore
tb.db_close()
